In [1]:
import asyncio
import pandas as pd
from datetime import datetime, time, timedelta
from zoneinfo import ZoneInfo
from alpaca.data.live.stock import StockDataStream
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetCalendarRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.enums import Adjustment

from IPython.display import clear_output


# Importing the modules themselves for reloading
import TradePosition
import TouchArea
import LiveTrader

# Reloading the modules to apply any changes
import importlib
importlib.reload(TradePosition)
importlib.reload(TouchArea)
importlib.reload(LiveTrader)


from TradePosition import TradePosition, SubPosition, export_trades_to_csv
from TouchArea import TouchArea, TouchAreaCollection
from LiveTrader import LiveTrader

import os, toml
from dotenv import load_dotenv

STANDARD_DATETIME_STR = '%Y-%m-%d %H:%M:%S'
ROUNDING_DECIMAL_PLACES = 10  # Choose an appropriate number of decimal places

load_dotenv(override=True)
livepaper = os.getenv('LIVEPAPER')
config = toml.load('../config.toml')

# Replace with your Alpaca API credentials
API_KEY = config[livepaper]['key']
API_SECRET = config[livepaper]['secret']


debug = True

def debug_print(*args, **kwargs):
    if debug:
        print(*args, **kwargs)

In [2]:
async def main():
    symbol = "AAPL"
    initial_balance = 10000
    trader = LiveTrader(API_KEY, API_SECRET, symbol, initial_balance)
    
    run_task = asyncio.create_task(trader.run())
    
    try:
        start_time = asyncio.get_event_loop().time()
        while True:
            if trader.is_receiving_data():
                print("Receiving data successfully.")
                break
            
            if asyncio.get_event_loop().time() - start_time > 240:
                raise TimeoutError("No data received within 240 seconds.")
            
            await asyncio.sleep(10)  # Check every 10 seconds
        
        # If we've reached here, we're receiving data. Let the trader run indefinitely.
        print("Trader is now running indefinitely.")
        await run_task
        
    except TimeoutError as e:
        print(f"{e} This is normal outside of market hours for IEX feed.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        run_task.cancel()
        try:
            await run_task
        except asyncio.CancelledError:
            pass
        await trader.close()
        print("Trader stopped.")


In [4]:
import tracemalloc
tracemalloc.start()


import logging
logging.getLogger('alpaca').setLevel(logging.DEBUG)

# Run the main function
clear_output(wait=True)  # Clear previous output

# Check if we're already in an event loop
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    # If we're not in an event loop, create one and run our main function
    asyncio.run(main())
else:
    # If we're already in an event loop, create a task
    loop.create_task(main())

asdf
asdf
asdf
